In [1]:
import pandas as pd
import numpy as np

In [2]:
#split training testing variation

In [13]:
original_ds = pd.read_csv("chicago_listings.csv")

In [14]:
import random
random.seed(123)

In [15]:
!mkdir split_variation

mkdir: cannot create directory ‘split_variation’: File exists


In [20]:
random.seed(123)

for x in range(10):
    !mkdir split_variation/split_$x
    sample_test_set = random.sample(original_ds.drop_duplicates("id",keep="first").id.values.tolist(),int(original_ds.shape[0] * .6))
    sample_train_set = set(original_ds.drop_duplicates("id",keep="first").id.values.tolist()) - set(sample_test_set)
    test_set = original_ds[original_ds.id.apply(lambda x: x in sample_test_set)]
    train_set = original_ds[original_ds.id.apply(lambda x: x in sample_train_set)]
    test_set_index = test_set.index.tolist()
    random.shuffle(test_set_index)
    train_all = original_ds[original_ds.id.apply(lambda x: x not in sample_test_set)]
    train_set.to_csv(f"split_variation/split_{x}/train_original.csv",index=False)
    test_set.to_csv(f"split_variation/split_{x}/test.csv",index=False)

In [21]:
# add noise on text based on prob
def add_noise_text(text,prob=.3,replace=True,seed=None):
    len_text = len(text)
    len_noise = round(prob*len_text)
    chars = [chr(c) for c in range(ord('a'),ord('z'))]
    chars += [chr(c) for c in range(ord('A'),ord('Z'))]
    chars += [" "]
    list_text = list(text)
    for x in range(len_noise):
        pos = random.sample(range(len_text),1)[0]
        if replace:
            list_text[pos] = random.sample(chars,1)[0]
        else:
            list_text.insert(pos,random.sample(chars,1)[0])
    #print(list_text)
    return "".join(list_text)

#test_char = add_noise_text("hello world")

In [39]:
def add_duplicate_id(airbnb_pd,prob,perturbed_id=[]):
    airbnb_ad = airbnb_pd.copy()
    airbnb_id = airbnb_pd.id.values.tolist()
    duplicate_id_prob = prob
    duplicate_id = random.sample(airbnb_id,int(duplicate_id_prob*len(airbnb_id)))

    inconsistent_id_prob = .3
    inconsistent_id = random.sample(duplicate_id,int(inconsistent_id_prob*len(duplicate_id)))
    
    airbnb_id_group = airbnb_pd.groupby(["id"])
    for d in duplicate_id:
        d_row = airbnb_id_group.get_group(d).copy()
        #print(d_row)
        if d in inconsistent_id:
            # randomize delete some columns to make the data inconsistent
            column_set = set(d_row.columns) - {"id","host_id","host_name"}
            column_r = random.sample(list(column_set),3)
            #print(column_r)
            for cr in column_r:
                d_row[cr] = ""
            #print(d_row)
            #break
        airbnb_ad = pd.concat([airbnb_ad,d_row])
        perturbed_id.append(d)
        #break
        
    
    return airbnb_ad

In [83]:
def add_inconsistent_host_name(airbnb_pd,prob,perturbed_id=[]):
    # host_id has functional dependency with host name
    airbnb_ad = airbnb_pd.copy()
    airbnb_id = airbnb_pd.id.values.tolist()
    
    inconsistent_host_name_prob = prob

    airbnb_group_host = airbnb_pd.groupby(["host_id","host_name"])
    airbnb_host_count = airbnb_group_host.count()[["id"]].sort_values("id",ascending=False).reset_index()
    airbnb_host_count[airbnb_host_count.id>1][["host_id","host_name"]]
    len_airbnb_host = len(airbnb_host_count[airbnb_host_count.id>1][["host_id","host_name"]].values.tolist())
    inconsistent_host = random.sample(airbnb_host_count[airbnb_host_count.id>1][["host_id","host_name"]].values.tolist(),int(inconsistent_host_name_prob*len_airbnb_host))
    
    for ih in inconsistent_host:
        ghost = airbnb_group_host.get_group(tuple(ih))
        #print(ghost)
        # get one sample
        isample = random.sample(list(ghost.index),1)
        or_name = airbnb_ad.iloc[isample]["host_name"].values[0]
        #airbnb_ad.loc[isample,"host_name"] = or_name + or_name
        airbnb_ad.loc[isample,"host_name"] = add_noise_text(or_name)
        #print(or_name)
        #break
    return airbnb_ad

In [84]:
def add_neighbourhood(airbnb_pd,prob):
    # random sample neighbourhood
    import numpy as np
    airbnb_ad = airbnb_pd.copy()
    neighbour_prob = prob
    num_row = airbnb_ad.shape[0]
    neighbour_noise = random.sample(list(airbnb_ad.index),round(neighbour_prob*num_row))

    for ih in neighbour_noise:
        rep_val = airbnb_ad.loc[ih,["neighbourhood"]].values[0]
        rep_val = rep_val[0] if type(rep_val) == np.ndarray else rep_val
        airbnb_ad.loc[ih,"neighbourhood"] = add_noise_text(rep_val)
    return airbnb_ad

In [85]:
# add noise on text based on prob
def add_noise_removed_period(text,replace=True,seed=None):
    return text.replace(".","")

#test_char = add_noise_text("hello world")

In [86]:
def add_latitude(airbnb_pd,prob):
    # random sample neighbourhood
    airbnb_ad = airbnb_pd.copy()

    inconsistent_lat_prob = prob / 2 

    num_row = airbnb_ad.shape[0]
    lat_noise = random.sample(list(airbnb_ad.index),round(inconsistent_lat_prob*num_row))

    for ih in lat_noise:
        rep_val = airbnb_ad.loc[ih,["latitude"]].values[0]
        rep_val = rep_val[0] if type(rep_val) == np.ndarray else rep_val
        airbnb_ad.loc[ih,"latitude"] = add_noise_text(str(rep_val),replace=False)

    inconsistent_lat_prob = prob/2
    lat_noise = random.sample(list(airbnb_ad.index),round(inconsistent_lat_prob*num_row))
    for ih in lat_noise:
        rep_val = airbnb_ad.loc[ih,["latitude"]].values[0]
        rep_val = rep_val[0] if type(rep_val) == np.ndarray else rep_val
        airbnb_ad.loc[ih,"latitude"] = add_noise_removed_period(str(rep_val),replace=False)
    
    return airbnb_ad

In [87]:
def add_longitude(airbnb_pd,prob):
    # random sample neighbourhood
    airbnb_ad = airbnb_pd.copy()

    inconsistent_lat_prob = prob/2

    num_row = airbnb_ad.shape[0]
    lat_noise = random.sample(list(airbnb_ad.index),round(inconsistent_lat_prob*num_row))

    for ih in lat_noise:
        rep_val = airbnb_ad.loc[ih,["longitude"]].values[0]
        rep_val = rep_val[0] if type(rep_val) == np.ndarray else rep_val
        airbnb_ad.loc[ih,"longitude"] = add_noise_text(str(rep_val),replace=False)


    inconsistent_lat_prob = prob/2
    lat_noise = random.sample(list(airbnb_ad.index),round(inconsistent_lat_prob*num_row))
    for ih in lat_noise:
        rep_val = airbnb_ad.loc[ih,["longitude"]].values[0]
        rep_val = rep_val[0] if type(rep_val) == np.ndarray else rep_val
        airbnb_ad.loc[ih,"longitude"] = add_noise_removed_period(str(rep_val),replace=False)

    return airbnb_ad

In [88]:
def add_minimum_nights(airbnb_pd,prob):
    # random sample neighbourhood
    airbnb_ad = airbnb_pd.copy()

    inconsistent_nights_prob = prob

    num_row = airbnb_ad.shape[0]
    lat_noise = random.sample(list(airbnb_ad.index),round(inconsistent_nights_prob*num_row))

    for ih in lat_noise:
        rep_val = airbnb_ad.loc[ih,["minimum_nights"]].values[0]
        rep_val = rep_val[0] if type(rep_val) == np.ndarray else rep_val
        airbnb_ad.loc[ih,"minimum_nights"] = add_noise_text(str(rep_val),replace=False)

    return airbnb_ad

In [89]:
def add_reviews(airbnb_pd,prob):
    # random sample neighbourhood
    airbnb_ad = airbnb_pd.copy()

    inconsistent_reviews_prob = prob

    num_row = airbnb_ad.shape[0]
    lat_noise = random.sample(list(airbnb_ad.index),round(inconsistent_reviews_prob*num_row))

    for ih in lat_noise:
        rep_val = airbnb_ad.loc[ih,["number_of_reviews"]].values[0]
        rep_val = rep_val[0] if type(rep_val) == np.ndarray else rep_val
        airbnb_ad.loc[ih,"number_of_reviews"] = add_noise_text(str(rep_val),replace=False)
        
    return airbnb_ad

In [90]:
from datetime import datetime

# add noise on text based on prob
def add_noise_change_date(text,target_date,source_date="%m/%d/%y",replace=True,seed=None):
    try:
        temp_date = datetime.strptime(str(text),"%m/%d/%y")
        return datetime.strftime(temp_date,target_date)
    except BaseException as ex:
        #print(ex)
        return text

#test_char = add_noise_text("hello world")
#airbnb_ad.last_review.apply(lambda x: datetime.strptime(x,"%m/%d/%y") if (type(x)==str)&(len(str(x))>0) else x)

In [91]:
def add_last_review(airbnb_pd,prob):
    # random sample neighbourhood
    airbnb_ad = airbnb_pd.copy()

    inconsistent_date_prob = prob / 2

    num_row = airbnb_ad.shape[0]
    review_noise = random.sample(list(airbnb_ad.index),round(inconsistent_date_prob*num_row))

    for ih in review_noise:
        rep_val = airbnb_ad.loc[ih,["last_review"]].values[0]
        rep_val = rep_val[0] if type(rep_val) == np.ndarray else rep_val
        airbnb_ad.loc[ih,"last_review"] = add_noise_change_date(rep_val,target_date="%B %d, %Y")

    review_noise = random.sample(list(airbnb_ad.index),round(inconsistent_date_prob*num_row))

    for ih in review_noise:
        rep_val = airbnb_ad.loc[ih,["last_review"]].values[0]
        rep_val = rep_val[0] if type(rep_val) == np.ndarray else rep_val
        airbnb_ad.loc[ih,"last_review"] = add_noise_change_date(rep_val,target_date="%d-%m-%Y")
        
    return airbnb_ad

In [92]:
def add_room_type(airbnb_pd,prob):
    # random sample neighbourhood
    airbnb_ad = airbnb_pd.copy()

    room_type_noise = prob

    noise_room_type = ["Entire home","Apartment"]

    home_pd = airbnb_ad[airbnb_ad.room_type=="Entire home/apt"]

    num_row = home_pd.shape[0]
    room_type_noise = random.sample(list(home_pd.index),round(room_type_noise*num_row))

    for ih in room_type_noise:
        rep_val = airbnb_ad.loc[ih,["room_type"]].values[0]
        rep_val = rep_val[0] if type(rep_val) == np.ndarray else rep_val
        airbnb_ad.loc[ih,"room_type"] = random.sample(noise_room_type,1)[0]
        
    return airbnb_ad

In [93]:
import os

In [104]:
random.seed(77)
# perturbed for each train data based on noise variation 20, 30, 40, 50
noise_var = [5,10,15,20,25,30,35,40]
flags = ['id_flag',
     'host_id_flag',
     'neighbourhood_flag',
     'latitude_flag',
     'longitude_flag',
     'minimum_nights_flag',
     'number_of_reviews_flag',
     'last_review_flag',
     'room_type_flag']
for x in os.listdir("split_variation"):
    print(x)
    for y in noise_var:        
        print(y)
        airbnb_pd = pd.read_csv(f"split_variation/{x}/train_original.csv")
        noise_prob = y / 100
        airbnb_pd = add_inconsistent_host_name(airbnb_pd,noise_prob)
        airbnb_pd = add_neighbourhood(airbnb_pd,noise_prob)
        airbnb_pd = add_latitude(airbnb_pd,noise_prob)
        airbnb_pd = add_logitude(airbnb_pd,noise_prob)
        airbnb_pd = add_minimum_nights(airbnb_pd,noise_prob)
        airbnb_pd = add_reviews(airbnb_pd,noise_prob)
        airbnb_pd = add_last_review(airbnb_pd,noise_prob)
        airbnb_pd = add_room_type(airbnb_pd,noise_prob)
        airbnb_pd = add_duplicate_id(airbnb_pd,noise_prob)        
        for f in flags:
            airbnb_pd[f] = 0
        airbnb_pd.to_csv(f"split_variation/{x}/train_noise_{y}.csv",index=False)


split_4
5
10
15
20
25
30
35
40
split_7
5
10
15
20
25
30
35
40
split_0
5
10
15
20
25
30
35
40
split_9
5
10
15
20
25
30
35
40
split_6
5
10
15
20
25
30
35
40
split_8
5
10
15
20
25
30
35
40
split_2
5
10
15
20
25
30
35
40
split_3
5
10
15
20
25
30
35
40
split_5
5
10
15
20
25
30
35
40
split_1
5
10
15
20
25
30
35
40


In [37]:
!ls split_variation/split_1/

test.csv  train_original.csv
